### Pyspark Handling Missing Values
- Eliminar columnas
- Eliminar Filas
- Parámetros para el Dropping
- Imputar valores nulos con la media, la mediana y la moda

#### CREAMOS LA SESIÓN

In [1]:
from pyspark.sql import SparkSession
spark = SparkSession.builder.appName('Practise').getOrCreate()

your 131072x1 screen size is bogus. expect trouble
24/03/06 20:38:42 WARN Utils: Your hostname, DESKTOP-8EEJDSD resolves to a loopback address: 127.0.1.1; using 172.28.93.32 instead (on interface eth0)
24/03/06 20:38:42 WARN Utils: Set SPARK_LOCAL_IP if you need to bind to another address
Setting default log level to "WARN".
To adjust logging level use sc.setLogLevel(newLevel). For SparkR, use setLogLevel(newLevel).
24/03/06 20:38:45 WARN NativeCodeLoader: Unable to load native-hadoop library for your platform... using builtin-java classes where applicable
24/03/06 20:38:48 WARN Utils: Service 'SparkUI' could not bind on port 4040. Attempting port 4041.


In [2]:
spark

#### CARGAMOS EL DATAFRAME

In [3]:
df_pyspark = spark.read.csv('data/test2.csv', header=True, inferSchema=True)
df_pyspark.show()

+---------+----+----------+------+
|     Name| age|Experience|Salary|
+---------+----+----------+------+
|    Krish|  31|        10| 30000|
|Sudhanshu|  30|         8| 25000|
|    Sunny|  29|         4| 20000|
|     Paul|  24|         3| 20000|
|   Harsha|  21|         1| 15000|
|  Shubham|  23|         2| 18000|
|   Mahesh|NULL|      NULL| 40000|
|     NULL|  34|        10| 38000|
|     NULL|  36|      NULL|  NULL|
+---------+----+----------+------+



In [9]:
df_pyspark.printSchema()

root
 |-- Name: string (nullable = true)
 |-- age: integer (nullable = true)
 |-- Experience: integer (nullable = true)
 |-- Salary: integer (nullable = true)



In [4]:
df_pyspark.na.drop().show()

+---------+---+----------+------+
|     Name|age|Experience|Salary|
+---------+---+----------+------+
|    Krish| 31|        10| 30000|
|Sudhanshu| 30|         8| 25000|
|    Sunny| 29|         4| 20000|
|     Paul| 24|         3| 20000|
|   Harsha| 21|         1| 15000|
|  Shubham| 23|         2| 18000|
+---------+---+----------+------+



La función drop() tiene varios parámetros, entre ellos "how", con valores "any" y "all".
- how = 'any' elimina la columna si uno de los valores es nulo (any por defecto)
- how = 'all' elimina la columna si todos los valores son nulos

Otro parámetro es "thresh". Por ejemplo: thresh = 2, significa que se mantienen las filas con al menos 2 valores NO NULOS. 

In [5]:
df_pyspark.na.drop(how = 'any', thresh = 2).show()

+---------+----+----------+------+
|     Name| age|Experience|Salary|
+---------+----+----------+------+
|    Krish|  31|        10| 30000|
|Sudhanshu|  30|         8| 25000|
|    Sunny|  29|         4| 20000|
|     Paul|  24|         3| 20000|
|   Harsha|  21|         1| 15000|
|  Shubham|  23|         2| 18000|
|   Mahesh|NULL|      NULL| 40000|
|     NULL|  34|        10| 38000|
+---------+----+----------+------+



In [6]:
df_pyspark.na.drop(how = 'any', thresh = 3).show()

+---------+---+----------+------+
|     Name|age|Experience|Salary|
+---------+---+----------+------+
|    Krish| 31|        10| 30000|
|Sudhanshu| 30|         8| 25000|
|    Sunny| 29|         4| 20000|
|     Paul| 24|         3| 20000|
|   Harsha| 21|         1| 15000|
|  Shubham| 23|         2| 18000|
|     NULL| 34|        10| 38000|
+---------+---+----------+------+



In [7]:
df_pyspark.na.drop(how = 'any', thresh = 1).show()

+---------+----+----------+------+
|     Name| age|Experience|Salary|
+---------+----+----------+------+
|    Krish|  31|        10| 30000|
|Sudhanshu|  30|         8| 25000|
|    Sunny|  29|         4| 20000|
|     Paul|  24|         3| 20000|
|   Harsha|  21|         1| 15000|
|  Shubham|  23|         2| 18000|
|   Mahesh|NULL|      NULL| 40000|
|     NULL|  34|        10| 38000|
|     NULL|  36|      NULL|  NULL|
+---------+----+----------+------+



El último parámetro es "subset". Con este, podemos decirle que elimine las columnas con valores nulos en una columna determinada.

In [ ]:
df_pyspark.na.drop(how = 'any', subset = ['Experience']).show()

RELLENAR MISSING VALUES

In [8]:
df_pyspark.na.fill('Missing Value').show() 

# también se podría poner df_pyspark.fillna().
# Solo rellenará las columnas que comparten el tipo de valor. Si lo que queremos introducir es un string, solo lo sustituirá en aquellas
# columnas que sean string. Esto es igual con las columnas numéricas. 

+-------------+----+----------+------+
|         Name| age|Experience|Salary|
+-------------+----+----------+------+
|        Krish|  31|        10| 30000|
|    Sudhanshu|  30|         8| 25000|
|        Sunny|  29|         4| 20000|
|         Paul|  24|         3| 20000|
|       Harsha|  21|         1| 15000|
|      Shubham|  23|         2| 18000|
|       Mahesh|NULL|      NULL| 40000|
|Missing Value|  34|        10| 38000|
|Missing Value|  36|      NULL|  NULL|
+-------------+----+----------+------+



In [ ]:
# Distintos tipos en las columnas seleccionadas.

df_pyspark.na.fill({'Name':'Missing Value', 'Experience':10}).show()

#### IMPUTAR VALORES

Podemos sustituir los valores por el valor medio, mediana... de la siguiente forma:

In [10]:
from pyspark.ml.feature import Imputer

In [11]:
imputer_mean = Imputer(inputCols=['age', 'Experience', 'Salary'],
                    outputCols = ['{}_imputed'.format(c) for c in ['age', 'Experience', 'Salary']]
                    ).setStrategy('mean')

In [12]:
imputer_mean.fit(df_pyspark).transform(df_pyspark).show()

+---------+----+----------+------+-----------+------------------+--------------+
|     Name| age|Experience|Salary|age_imputed|Experience_imputed|Salary_imputed|
+---------+----+----------+------+-----------+------------------+--------------+
|    Krish|  31|        10| 30000|         31|                10|         30000|
|Sudhanshu|  30|         8| 25000|         30|                 8|         25000|
|    Sunny|  29|         4| 20000|         29|                 4|         20000|
|     Paul|  24|         3| 20000|         24|                 3|         20000|
|   Harsha|  21|         1| 15000|         21|                 1|         15000|
|  Shubham|  23|         2| 18000|         23|                 2|         18000|
|   Mahesh|NULL|      NULL| 40000|         28|                 5|         40000|
|     NULL|  34|        10| 38000|         34|                10|         38000|
|     NULL|  36|      NULL|  NULL|         36|                 5|         25750|
+---------+----+----------+-